In [1]:
import os
import time

from torchvision import transforms, models
from torch.utils.data import RandomSampler, DataLoader
import torch.nn as nn
import torch.optim as optim

from implementation import CNNClassifier, train_model
from loader import load_png_images

In [2]:
def prettify_duration(duration_in_seconds):
    s = duration_in_seconds % 60
    duration_in_seconds -= s
    duration_in_seconds //= 60

    m = duration_in_seconds % 60
    duration_in_seconds -= m
    duration_in_seconds //= 60

    return f"{duration_in_seconds}h {m}m {s}s"

In [3]:
PREFIX = "sample"

In [4]:
train_path = os.path.join(os.getcwd(), "data", PREFIX, "train")
valid_path = os.path.join(os.getcwd(), "data", PREFIX, "valid")
test_path = os.path.join(os.getcwd(), "data", PREFIX, "test")

In [5]:
transformations = [
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ], [
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomInvert(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomResizedCrop(size=32, scale=(0.8, 1.0)),
        transforms.GaussianBlur(11),
        transforms.ToTensor(),
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
        transforms.Normalize((0.5,), (0.5,))
    ], [
        transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ], [
        transforms.RandAugment(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]
]

In [6]:
for transformation in transformations:
    train_full, t_n = load_png_images(train_path, transform=transformation, batch_size=64)
    train_subset = RandomSampler(train_full.dataset, num_samples=500)
    train = DataLoader(train_full.dataset, sampler=train_subset, batch_size=64)
    
    valid_full, v_n = load_png_images(valid_path, transform=transformation, batch_size=2048)
    valid_subset = RandomSampler(valid_full.dataset, num_samples=500)
    valid = DataLoader(valid_full.dataset, sampler=valid_subset, batch_size=2048)

    m_models = [
        CNNClassifier(t_n),
        models.densenet121(weights="DenseNet121_Weights.IMAGENET1K_V1"),
        models.wide_resnet50_2(weights="Wide_ResNet50_2_Weights.DEFAULT")
    ]
    m_models[1].classifier = nn.Linear(m_models[1].classifier.in_features, t_n)
    m_models[2].fc = nn.Linear(m_models[2].fc.in_features, t_n)
    
    for model in m_models:
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        start = time.time()
        train_model(model, train, valid, criterion, optimizer, epochs=10)
        end = time.time()

        print(f"training took {prettify_duration(end - start)}")

Epoch 1/10, Loss: 0.7052, Validation Accuracy: 53.80%
Epoch 2/10, Loss: 0.6910, Validation Accuracy: 56.80%
Epoch 3/10, Loss: 0.6783, Validation Accuracy: 64.80%
Epoch 4/10, Loss: 0.6566, Validation Accuracy: 58.40%
Epoch 5/10, Loss: 0.6440, Validation Accuracy: 62.60%
Epoch 6/10, Loss: 0.6356, Validation Accuracy: 66.40%
Epoch 7/10, Loss: 0.6143, Validation Accuracy: 65.40%
Epoch 8/10, Loss: 0.6082, Validation Accuracy: 63.00%
Epoch 9/10, Loss: 0.6129, Validation Accuracy: 68.80%
Epoch 10/10, Loss: 0.6046, Validation Accuracy: 69.60%
training took 0.0h 2.0m 7.976621389389038s
Epoch 1/10, Loss: 0.6455, Validation Accuracy: 63.40%
Epoch 2/10, Loss: 0.6016, Validation Accuracy: 66.80%
Epoch 3/10, Loss: 0.5104, Validation Accuracy: 77.00%
Epoch 4/10, Loss: 0.5631, Validation Accuracy: 73.20%
Epoch 5/10, Loss: 0.5211, Validation Accuracy: 75.60%
Epoch 6/10, Loss: 0.5182, Validation Accuracy: 78.40%
Epoch 7/10, Loss: 0.4836, Validation Accuracy: 75.80%
Epoch 8/10, Loss: 0.5107, Validation A